In [2]:
import numpy as np
np.set_printoptions(precision=3) # 2 decimals when printing arrays
np.set_printoptions(suppress=True) # No scientific notation for small numbers
import matplotlib.pyplot as plt

from quantum_systems import ODQD, GeneralOrbitalSystem

from coupled_cluster.ccd.rhs_t import compute_t_2_amplitudes
from coupled_cluster.mix import AlphaMixer, DIIS
from coupled_cluster.ccd import CCD
from coupled_cluster.ccsd import CCSD

ModuleNotFoundError: No module named 'quantum_systems'

In [144]:
def pprint(tensor):
    print(tensor[:, :, 1, 0].real)

## Setting up the system

In [157]:
l_0 = 10                 # number of basis functions
grid_length = 10        # compute from x = -10 to x = 10 in 1D
num_grid_points = 2001
omega = 0.25            # strength of harmonic oscillator potential
n = 2                   # number of particles

odho = ODQD(l_0, grid_length, num_grid_points, a = 0.25, alpha = 1, potential = ODQD.HOPotential(omega))
system = GeneralOrbitalSystem(n = n, basis_set=odho)

### CCSD

In [162]:
ccsd = CCSD(system, mixer = DIIS, verbose=False)
print(f"Ground state energy: {ccsd.compute_energy().real:.6}")

Ground state energy: 0.644717


In [163]:
pprint(ccsd.t_2)

[[-0.     0.685 -0.    -0.    -0.     0.234 -0.    -0.   ]
 [-0.685 -0.     0.    -0.    -0.234 -0.     0.    -0.   ]
 [-0.    -0.    -0.     0.303 -0.     0.    -0.     0.134]
 [ 0.    -0.    -0.303 -0.    -0.    -0.    -0.134 -0.   ]
 [-0.     0.234 -0.     0.    -0.     0.158 -0.    -0.   ]
 [-0.234 -0.    -0.    -0.    -0.158 -0.     0.    -0.   ]
 [-0.    -0.    -0.     0.134 -0.    -0.    -0.     0.094]
 [ 0.    -0.    -0.134 -0.     0.    -0.    -0.094 -0.   ]]


In [160]:

ccsd.compute_ground_state()
print(f"Ground state energy: {ccsd.compute_energy():.6}")

Ground state energy: (0.826268+0j)


## Coupled cluster calculation

In [126]:
ccd = CCD(system, mixer = DIIS, verbose=False)
#ccd.setup_t_mixer(np=np)

In [129]:
conv_tol = 1e-7
t_kwargs = dict(tol=conv_tol)
l_kwargs = dict(tol=conv_tol)
print(f"Ground state energy: {ccd.compute_energy():.6}")
ccd.compute_ground_state(t_kwargs=t_kwargs, l_kwargs=l_kwargs)
print(f"Ground state energy: {ccd.compute_energy():.6}")
#energy = ccd.compute_one_body_expectation_value(system.h) + 0.5 * ccd.compute_two_body_expectation_value(system.u)
#print(energy)

Ground state energy: (1.05156+0j)
Ground state energy: (1.05156+0j)


In [99]:
ccd.rhs_t_2.fill(0)
compute_t_2_amplitudes(ccd.f, ccd.u, ccd.t_2, ccd.o, ccd.v, out = ccd.rhs_t_2, np = np)
direction_vector_1 = np.divide(ccd.rhs_t_2, ccd.d_t_2)
trial_vector_1 = np.copy(ccd.t_2)
error_vector_1 = np.copy(ccd.rhs_t_2)

ccd.t_2 = ccd.t_2_mixer.compute_new_vector(trial_vector_1, direction_vector_1, error_vector_1)

In [100]:
pprint(trial_vector_1)
pprint(direction_vector_1)
pprint(error_vector_1)

[[-0.       0.6855  -0.      -0.      -0.       0.23397 -0.      -0.     ]
 [-0.6855  -0.       0.      -0.      -0.23397 -0.       0.      -0.     ]
 [-0.      -0.      -0.       0.30334 -0.       0.      -0.       0.13369]
 [ 0.      -0.      -0.30334 -0.      -0.      -0.      -0.13369 -0.     ]
 [-0.       0.23397 -0.       0.      -0.       0.15818 -0.      -0.     ]
 [-0.23397 -0.      -0.      -0.      -0.15818 -0.       0.      -0.     ]
 [-0.      -0.      -0.       0.13369 -0.      -0.      -0.       0.09357]
 [ 0.      -0.      -0.13369 -0.       0.      -0.      -0.09357 -0.     ]]
[[-0.      -0.64469 -0.       0.      -0.      -0.13621 -0.       0.     ]
 [ 0.64469 -0.      -0.      -0.       0.13621 -0.      -0.      -0.     ]
 [-0.       0.      -0.      -0.39945 -0.      -0.      -0.      -0.15562]
 [-0.      -0.       0.39945 -0.       0.      -0.       0.15562 -0.     ]
 [-0.      -0.13621 -0.      -0.      -0.      -0.23933 -0.       0.     ]
 [ 0.13621 -0.       0. 

In [77]:
ccd.rhs_t_2.fill(0)
compute_t_2_amplitudes(ccd.f, ccd.u, ccd.t_2, ccd.o, ccd.v, out = ccd.rhs_t_2, np = np)
direction_vector_2 = np.divide(ccd.rhs_t_2, ccd.d_t_2)
trial_vector_2 = ccd.t_2
error_vector_2 = ccd.rhs_t_2

ccd.t_2_mixer.compute_new_vector(trial_vector_2, direction_vector_2, error_vector_2)[:, :, 1, 0].real

array([[ 0.   ,  0.47 ,  0.   , -0.   ,  0.   ,  0.172],
       [-0.47 ,  0.   ,  0.   ,  0.   , -0.172,  0.   ],
       [ 0.   , -0.   ,  0.   ,  0.149,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.149,  0.   , -0.   ,  0.   ],
       [ 0.   ,  0.172,  0.   ,  0.   ,  0.   ,  0.018],
       [-0.172,  0.   , -0.   ,  0.   , -0.018,  0.   ]])

In [78]:
pprint(trial_vector_2)
pprint(direction_vector_2)
pprint(error_vector_2)

[[ 0.     0.192  0.    -0.     0.     0.142]
 [-0.192  0.     0.     0.    -0.142  0.   ]
 [ 0.    -0.     0.    -0.031  0.    -0.   ]
 [ 0.     0.     0.031  0.     0.     0.   ]
 [ 0.     0.142  0.    -0.     0.    -0.103]
 [-0.142  0.     0.     0.     0.103  0.   ]]
[[-0.     0.571 -0.    -0.    -0.     0.06 ]
 [-0.571 -0.     0.    -0.    -0.06  -0.   ]
 [-0.    -0.    -0.     0.37  -0.     0.   ]
 [ 0.    -0.    -0.37  -0.    -0.    -0.   ]
 [-0.     0.06  -0.     0.    -0.     0.248]
 [-0.06  -0.    -0.    -0.    -0.248 -0.   ]]
[[ 0.    -0.309  0.     0.     0.    -0.049]
 [ 0.309  0.    -0.     0.     0.049  0.   ]
 [ 0.     0.     0.    -0.281  0.    -0.   ]
 [-0.     0.     0.281  0.     0.     0.   ]
 [ 0.    -0.049  0.    -0.     0.    -0.265]
 [ 0.049  0.     0.     0.     0.265  0.   ]]


In [ ]:
ccd.setup_t_mixer(np=np)
pprint(ccd.t_2_mixer.compute_new_vector(
    trial_vector_1, direction_vector_1, error_vector_1))

In [ ]:
pprint(ccd.t_2_mixer.compute_new_vector(
    trial_vector_2, direction_vector_2, error_vector_2))

In [ ]:
mixer = DIIS(np = np)

In [ ]:
class AlphaMixer:
    """Basic mixer class
    Parameters
    ----------
    theta : float
        Mixing parameter. Must be in [0, 1]
    np : module
        Matrix library to be used, e.g., numpy, cupy, etc.
    """

    def __init__(self, theta=0.1, np=None):
        assert 0 <= theta <= 1, "Mixing parameter theta must be in [0, 1]"

        self.theta = theta

    def compute_new_vector(self, trial_vector, direction_vector, error_vector):
        """Compute new trial vector for mixing with full right hand side.
        See T. Helgaker's book "Molecular Electron-Structure Theory" equations
        (13.4.3), (13.4.6) and (13.4.10).
        Parameters
        ----------
        trial_vector : np.array
            Inital vector for mixing.
        direction_vector : np.array
            Vector to be addet to trial_vector.
        error_vector : np.array
            Not used in alpha mixer, for DIIS mixer.
        Returns
        -------
        np.array
            New mixed vector.
        """
        new_trial = trial_vector + direction_vector

        return (1 - self.theta) * new_trial + self.theta * trial_vector

    def clear_vectors(self):
        pass
    
class DIIS(AlphaMixer):
    """Direct Inversion in Iterative Subspace (DIIS)
    General vector mixing class to accelerate quasi-Newton
    using direct inversion of iterative space.
    Code inherited from Simen Kvaal.
    Parameters
    ----------
    np : module
        Matrix library to be used, e.g., numpy, cupy, etc.
    num_vecs : int
        Number of vectors to keep in memory. Default is ``10``.
    """

    def __init__(self, np, num_vecs=10):
        self.np = np
        self.num_vecs = num_vecs
        self.stored = 0

        self.trial_vectors = [0] * self.num_vecs
        self.direction_vectors = [0] * self.num_vecs
        self.error_vectors = [0] * self.num_vecs

    def compute_new_vector(self, trial_vector, direction_vector, error_vector):
        """DIIS mixing scheme
        Parameters
        ----------
        trial_vector : np.array
            Inital vector for mixing.
        direction_vector : np.array
            Vector to be addet to trial_vector.
        error_vector : np.array
        Returns
        -------
        np.array
            New mixed vector
        """

        np = self.np

        new_pos = self.stored % self.num_vecs
        self.stored += 1

        self.trial_vectors[new_pos] = trial_vector.ravel()
        self.direction_vectors[new_pos] = direction_vector.ravel()
        self.error_vectors[new_pos] = error_vector.ravel()

        b_dim = self.stored if self.stored < self.num_vecs else self.num_vecs

        b_vec = np.zeros(b_dim + 1, dtype=trial_vector.dtype)
        b_mat = np.zeros((b_dim + 1, b_dim + 1), dtype=trial_vector.dtype)

        for i in range(b_dim):
            for j in range(i + 1):
                b_mat[i, j] = np.dot(
                    self.error_vectors[i], self.error_vectors[j]
                )

                if i != j:
                    b_mat[j, i] = b_mat[i, j]

            b_mat[i, b_dim] = -1.0
            b_mat[b_dim, i] = -1.0

        b_vec[b_dim] = -1.0
        pre_condition = np.zeros_like(b_vec)
        
        if np.any(np.diag(b_mat)[:-1] <= 0):
            pre_condition[:-1] = 1
        else:
            pre_condition[:-1] += np.power(np.diag(b_mat)[:-1], -0.5)

        pre_condition[b_dim] = 1

        for i in range(b_dim + 1):
            for j in range(b_dim + 1):
                b_mat[i, j] *= pre_condition[i] * pre_condition[j]
        
        weights = -np.linalg.pinv(b_mat)[b_dim]
        weights[:-1] *= pre_condition[:-1]

        new_trial_vector = np.zeros_like(self.trial_vectors[new_pos])

        for i in range(b_dim):
            new_trial_vector += weights[i] * (
                self.trial_vectors[i] + self.direction_vectors[i]
            )

        return new_trial_vector.reshape(trial_vector.shape)

    def clear_vectors(self):
        """
        Delete all stored vectors and start fresh.
        """

        self.trial_vectors = [0] * self.num_vecs
        self.direction_vectors = [0] * self.num_vecs
        self.error_vectors = [0] * self.num_vecs

        self.stored = 0

In [ ]:
ccsd = CCSD(system, mixer = DIIS, verbose=False)
conv_tol = 1e-4
t_kwargs = dict(tol=conv_tol)
l_kwargs = dict(tol=conv_tol)

print(f"Ground state energy: {ccsd.compute_energy():.4}")
ccsd.compute_ground_state(t_kwargs=t_kwargs, l_kwargs=l_kwargs)
print(f"Ground state energy: {ccsd.compute_energy():.4}")